In [ ]:
import os
os.environ['DJANGO_SETTINGS_MODULE'] = 'ClusterCast.settings'
import test_utils
import django
django.setup()
import sys
sys.path.append("/home/ajp031/StockDeepLearning/ClusterCast/ClusterCast")
from asgiref.sync import sync_to_async
from importlib import reload
import ClusterPipeline.models.ClusterProcessing as cp
from copy import deepcopy
import numpy as np
from tensorflow.keras.backend import clear_session

In [ ]:
reload(test_utils)

In [ ]:
group_ids = [40]
groups = await test_utils.extract_cluster_groups(group_ids)

In [ ]:
clusters = await test_utils.get_all_clusters(groups)

In [ ]:
cluster_of_interest_id = 444
cluster_of_interest = [cluster for cluster in clusters if cluster.id == cluster_of_interest_id][0]

In [ ]:
features = groups[0].group_params.training_features

features

In [ ]:
cur_group = groups[0]

In [ ]:
# cur_group = await test_utils.recreate_group(cur_group)

In [ ]:

X_train, y_train, X_test, y_test = cur_group.get_3d_array() 
X_train = X_train.copy()[::-1]
y_train = y_train.copy()[::-1]
X_test = X_test.copy()[::-1]
y_test = y_test.copy()[::-1]
cluster_features = cur_group.group_params.cluster_features
X_feature_dict = cur_group.group_params.X_feature_dict
y_feature_dict = cur_group.group_params.y_feature_dict
training_dict = {}
training_dict['0'] = (0,(X_train, y_train, X_test, y_test))
print(training_dict['0'][1][0].shape)
print(training_dict['0'][1][1].shape)
print(training_dict['0'][1][2].shape)
print(training_dict['0'][1][3].shape)

In [ ]:
print(len(X_feature_dict))
print(len(cur_group.group_params.X_cols))

In [ ]:
print(X_feature_dict)

In [ ]:
next_clusters = test_utils.cluster_a_group(cur_group,training_dict['0'][1],4)
next_label = 2
training_dict['1'] = next_clusters[next_label]

In [ ]:
for i in range(len(next_clusters)): 
    print(next_clusters[i][0].shape)
    plot = test_utils.visualize_cluster_centroid(next_clusters[i][0],cluster_features)
    plot.show()

In [ ]:
next_clusters = test_utils.cluster_a_group(cur_group,training_dict['1'][1],2)
for i in range(len(next_clusters)): 
    print(next_clusters[i][0].shape)
    plot = test_utils.visualize_cluster_centroid(next_clusters[i][0],cluster_features)
    plot.show()
next_label = 0
training_dict['2'] = next_clusters[next_label]

In [ ]:
next_clusters = test_utils.cluster_a_group(cur_group,training_dict['2'][1],3)
for i in range(len(next_clusters)): 
    print(next_clusters[i][0].shape)
    plot = test_utils.visualize_cluster_centroid(next_clusters[i][0],cluster_features)
    plot.show()
next_label = 1
training_dict['3'] = (next_clusters[next_label])

In [ ]:
import random
target_features = [] 
# target_features += ['pctChgclose{}_target'.format(i) for i in range(-14, 0) ]
# target_features += ['pctChgclose-0_target']
target_features += ['pctChgclose+{}_target'.format(i) for i in range(1, 16) ]
training_features = random.sample(features, 20)
training_features += ['pctChgclose']


In [ ]:
print(training_features)

In [ ]:
reload(test_utils)

In [ ]:
print(target_features)
print(len(target_features))

In [ ]:
centroid =  training_dict['0'][0]
X_train, y_train, X_test, y_test = training_dict['0'][1]

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(y_feature_dict)

In [ ]:
bb_index = X_feature_dict['pctChgclose']
X_train[:,:,bb_index]

In [ ]:
# cur_group.run_clustering()
# all_centroids = cur_group.cluster_centers

# closted_centroid_label = test_utils.find_closest_centroid(centroid,all_centroids)
# print(closted_centroid_label)



In [ ]:
# print(cur_group.cluster_centers.shape)   


In [ ]:
# print(closted_centroid_label)
# test_utils.visualize_cluster_centroid(centroid,cluster_features).show()
# test_utils.visualize_cluster_centroid(all_centroids[closted_centroid_label],cluster_features).show()


In [ ]:
print(len(cur_group.clusters))

In [ ]:

X_train_filtered = test_utils.filter_by_features(X_train, training_features, X_feature_dict)
X_test_filtered = test_utils.filter_by_features(X_test, training_features, X_feature_dict)
y_train_filtered = test_utils.filter_y_by_features(y_train, target_features, y_feature_dict)
y_test_filtered = test_utils.filter_y_by_features(y_test, target_features, y_feature_dict)
print(X_train_filtered.shape)
print(y_train_filtered.shape)
print(X_test_filtered.shape)
print(y_test_filtered.shape)

In [ ]:
print(X_train_filtered[0,0,:])

In [ ]:
training_attention_model, test_attention_model = test_utils.create_attention_model(input_steps=X_train.shape[1],output_steps=len(target_features),features=len(training_features))

In [ ]:
decoder_weights = test_utils.save_decoder_initial_weights(training_attention_model)

In [ ]:
trainined_attention_model = test_utils.train_model(training_attention_model, (X_train, y_train, X_test, y_test), training_features, 
                                                   target_features, X_feature_dict, y_feature_dict,
                                                   epochs = 100, batch_size = 128, lr = 0.001, early_stopping_patience = 15,
                                                   loss = 'mse', shuffle = False)

In [ ]:
num_days = 15
attention_model_accuracy, attention_results, predicted_y, attention_weights = test_utils.eval_model(X_test_filtered, y_test_filtered, trainined_attention_model,num_days, test_model = False)

In [ ]:
for layer in trainined_attention_model.layers:
    if 'freeze' in layer.name:
        layer.trainable = False
        continue
    if 'input' in layer.name:
        continue
    layer.set_weights(decoder_weights[layer.name])

In [ ]:
reload(test_utils)

In [ ]:
print(attention_model_accuracy)
test_utils.visualize_future_distribution(attention_results)

In [ ]:
centroid =  training_dict['2'][0]
X_train_smaller, y_train_smaller, X_test_smaller, y_test_smaller= training_dict['2'][1]
# X_train_larger = np.concatenate((X_train, X_train_larger), axis=0)
# y_train_larger = np.concatenate((y_train, y_train_larger), axis=0)
# X_test_larger = np.concatenate((X_test, X_test_larger), axis=0)
# y_test_larger = np.concatenate((y_test, y_test_larger), axis=0)

# X_train_larger = np.concatenate((X_train, X_train_larger), axis=0)
# y_train_larger = np.concatenate((y_train, y_train_larger), axis=0)
# X_test_larger = np.concatenate((X_test, X_test_larger), axis=0)
# y_test_larger = np.concatenate((y_test, y_test_larger), axis=0)

X_train_smaller_filtered = test_utils.filter_by_features(X_train_smaller, training_features, X_feature_dict)
X_test_smaller_filtered = test_utils.filter_by_features(X_test_smaller, training_features, X_feature_dict)
y_train_smaller_filtered = test_utils.filter_y_by_features(y_train_smaller, target_features, y_feature_dict)
y_test_smaller_filtered = test_utils.filter_y_by_features(y_test_smaller, target_features, y_feature_dict)



print(X_train_smaller_filtered.shape)
print(y_train_smaller_filtered.shape)
print(X_test_smaller_filtered.shape)
print(y_test_smaller_filtered.shape)

In [ ]:
larger_train_model = test_utils.train_model(trainined_attention_model, (X_train_smaller, y_train_smaller, X_test_smaller, y_test_smaller), training_features, 
                                                   target_features, X_feature_dict, y_feature_dict,
                                                   epochs = 150, batch_size = 8, lr = 0.0001, early_stopping_patience = 30,
                                                   loss = 'mse')

In [ ]:
large_data_set_accuracy, large_data_set_results, predicted_y, attention_weights = test_utils.eval_model(X_test_smaller_filtered, y_test_smaller_filtered, test_attention_model, num_days)

In [ ]:
benchmark_model = test_utils.create_attention_model(input_steps=X_train.shape[1],output_steps=len(target_features),features=len(training_features))
benchark_model = test_utils.train_model(benchmark_model, (X_train_smaller, y_train_smaller, X_test_smaller, y_test_smaller), training_features,
                                        target_features, X_feature_dict, y_feature_dict,
                                        epochs = 250, batch_size = 32, lr = 0.001, early_stopping_patience = 30,
                                        loss = 'mse')

In [ ]:
# benchark_model = test_utils.train_model(benchmark_model, (X_train_smaller, y_train_smaller, X_test_smaller, y_test_smaller), training_features,
#                                         target_features, X_feature_dict, y_feature_dict,
#                                         epochs = 250, batch_size = 16, lr = 0.0001, early_stopping_patience = 30,
#                                         loss = test_utils.custom_profit_loss_percent_change)

In [ ]:
bench_model_accuracy, bench_results = test_utils.eval_model(X_test_smaller_filtered, y_test_smaller_filtered, benchmark_model)

In [ ]:
print("FINE TUNED MODEL")
print(large_data_set_accuracy)
test_utils.visualize_future_distribution(large_data_set_results)

In [ ]:
print("BENCHMARK MODEL")
print(bench_model_accuracy)
test_utils.visualize_future_distribution(bench_results)

In [ ]:
clear_session()